# test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
import pandas as pd
import random
count=1
# what_type별로 질문 뽑기 및 각 질문의 what_type 저장
def get_random_questions_and_types(total_question, num_questions_per_type):
    random_questions = []
    what_type_list = []
    for what_type, question_counts in num_questions_per_type.items():
        questions_of_type = qa_df[qa_df['what_type'] == what_type]
        what_type = qa_df['what_type']

        save_user_answer = f'q{what_type}_{count}'

        #random_rows = random.sample(questions_of_type.index.tolist(), 10)


        # 실제 질문 수보다 요청된 샘플 크기가 큰 경우 조정
        #actual_num_questions = min(len(questions_of_type), question_counts)
        if actual_num_questions > 0:  # 질문이 존재하는 경우에만 처리
            random_rows = random.sample(range(len(total_question)), 10)
            for row in random_rows:
                 random_questions.append(questions_of_type.loc[row]['Question'])
                 what_type_list.append(what_type)


    return random_questions, what_type_list

In [ ]:
save_user_answer = pd.DataFrame(columns = ['name'])

In [ ]:
count = 1
qa_df = pd.DataFrame(columns = ['Question', 'Answer', 'pred', 'what_type'])

In [ ]:
total_question = pd.read_csv(r'/content/drive/MyDrive/MBTI_PRJ/Dataset/Data/total_question.csv')
num_questions_per_type = {1: 2, 2: 3, 3: 3, 4: 2}  # 예: what_type 1은 2개, what_type 2는 3개
random_questions, what_type_list = get_random_questions_and_types(total_question, num_questions_per_type)

random_questions

NameError: ignored

# 0.환경 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
pip install konlpy

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
pip install sentencepiece

In [ ]:
#훈련된 서브워드 토크나이저 모델 불러오기
import sentencepiece as spm

saved_path = '/content/drive/MyDrive/MBTI_PRJ/Dataset/save_model/tokenizer/m.model'
sp = spm.SentencePieceProcessor()
sp.Load(saved_path)#m.model 불러오기

In [ ]:
#훈련된 Bi-LSTM모델 불러오기
from keras.models import load_model
import tensorflow as tf
model_save_path = '/content/drive/MyDrive/MBTI_PRJ/Dataset/save_model/biLSTM_model/best_model.h5'
loaded_model = tf.keras.models.load_model(model_save_path)

# 모델 구조와 가중치가 올바르게 불러와졌는지 확인
loaded_model.summary()

# 1. 사용자 응답분석을 위한 함수 정의

In [ ]:
#Noun, Adjective, Verb, Adverb 단어만 추출
def extract_words(pos_list): #pos_list는 'pos_tagged'에 들어있는 word와 tag를 추출한다.
  target_tags = ['NNG', 'NNP', 'VV', 'VA', 'MAG', 'VA+ETM']
  return [word    for word,tag in pos_list   if tag in target_tags]

In [ ]:
#응답 분석
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

#extract word 추출
def tokenize_tagged(text):
  tagged_text = mecab.pos(text)#품사 태깅
  extracted_words = extract_words(tagged_text)#특정단어만 추출하여 저장
  return extracted_words

max_len=40


def predict_sentence(model, sentence, max_len):
    # 문장 전처리
    new_sentence = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sentence)
    new_sentence_morphs = tokenize_tagged(new_sentence)

    tokens_1 = sp.EncodeAsPieces(new_sentence)
    tokens_2 = sp.EncodeAsPieces(new_sentence_morphs)
    tokens_encode_1 = sp.EncodeAsIds(new_sentence)
    tokens_encode_2 = sp.EncodeAsIds(new_sentence_morphs)

    #중첩 리스트가 있을경우 평탄화
    if any(isinstance(i, list) for i in tokens_encode_2):
      tokens_encode_2 = [item for sublist in tokens_encode_2 for item in sublist]

    #패딩
    padded_sequences_1 = pad_sequences([tokens_encode_1], maxlen=max_len, padding='post')
    padded_sequences_2 = pad_sequences([tokens_encode_2], maxlen=max_len, padding='post')

    # 예측
    prediction_1 = model.predict(padded_sequences_1)
    prediction_2 = model.predict(padded_sequences_2)
    prediction_value_1 = prediction_1[0][0]
    prediction_value_2 = prediction_2[0][0]

    #예측값 계산
    result_mbti_value = (prediction_value_1 + prediction_value_2)/2.0
    print("  에측값 : ", result_mbti_value)

    return result_mbti_value


In [ ]:
#최종 판단 MBTI 출력 함수
def mbti_print_1(dataframe):
  global check_mbti

  result = qa_df['MBTI'].sum() / 2

  if result > 0.8: #1에 가까움
      print(f"   >> {result * 100:.2f}% 정도로 E 유형입니다.")
  elif result>0.2 and result<=0.8:
      check_mbti = True #재질문 위한 코드
  elif result <= 0.2: #0에 가까움
      print(f"   >> {100 - result * 100:.2f}% 정도로 I 유형입니다.")

In [ ]:
#추가 질문에 대한 MBTI 출력 함수
def mbti_print_2(dataframe):

  result = qa_df['MBTI'].sum() / 3

  if result > 0.8: #1에 가까움
      print(f"   >> {result * 100:.2f}% 정도로 E 유형입니다.")
  elif result>0.5 and result<=0.8:
      print(f"   >> {result * 100:.2f}% 정도로 E 유형입니다.")
  elif result>0.2 and result<=0.5:
      print(f"   >> {100 - result * 100:.2f}% 정도로 I 유형입니다.")
  elif result <= 0.2: #0에 가까움
      print(f"   >> {100 - result * 100:.2f}% 정도로 I 유형입니다.")

# 2. 사용자 질의응답
- 기존에 제작한 질문 데이터를 바탕으로 사용자에게 랜덤으로 2개의 질문을 한다.
- 사용자는 매 질문에 응답한다.
- 사용자 응답을 기반으로 사전 학습된 모델이 사용자의 MBTI를 판단한다.
- 질문 후, 정확한 예측이 이뤄지지 않았다고 판단되면 추가 질문을 하도록 한다.</br></br>
++ 해당 코드는 한 개의 MBTI 유형에 대해서 예측을 잘 하는지 여부를 판단하기 위한 코드이다. 나머지 MBTI에 대해서도 예측을 원한다면 코드 작성이 추가롤 필요하다.</br>
++ 추가 코드 링크는 제작자의 깃허브 'web/chat.py'에서 확인 가능하다.

In [ ]:
#질문 데이터셋 불러오기
import pandas as pd
question = pd.read_csv('/content/drive/MyDrive/MBTI_PRJ/Dataset/answer_analysis_data/question.csv')

In [ ]:
#데이터 프레임 초기화
qa_df = pd.DataFrame(columns = ['Question', 'Answer', 'pred', 'type'])

def initialize_chat(index):
   global qa_df
   qa_df = pd.DataFrame(columns = ['Question', 'Answer', 'pred', 'type'])
   random_question = random.sample(question['Question'].tolist(), 3)#3개 랜덤하게 뽑기
   qa_df['Question'] = random_question
   new_question = get_question(index)
   return new_question

def get_question(index):#단일질문 반환
    return qa_df.iloc[index]['Question']

In [ ]:
#질의응답 함수
def Question_Answer(n,dataframe):
  print(f'질문 {n+1} : ', qa_df['Question'][n])
  answer = (input('  당신의 답변 >> '))
  qa_df['Answer'][n]  = answer
  print('\n')

#### 사용자 응답기반 MBTI 예측

In [ ]:
#질문에 따른 응답입력
check_mbti = False

#실제 사용자 질의응답 단계
for n in range(2):
  #질의응답
  Question_Answer(n,qa_df)
  #예측 실행
  print('>> ', n+1, '번째 응답에 대한 MBTI 예측')
  MBTI = predict_sentence(loaded_model, str(qa_df['Answer'][n]), max_len)
  qa_df['MBTI'][n] = MBTI
  print('\n')

mbti_print_1(qa_df)


질문 1 :  새로운 사람들을 만났을 때 당신은 어떤 반응을 보이나요?
  당신의 답변 >> ㄴ


>>  1 번째 응답에 대한 MBTI 예측
1/1 [==============================] - 0s 29ms/step
  에측값 :  0.9710767865180969


질문 2 :  사람들과 함께 있을 때 어떤 기분이 드나요?
  당신의 답변 >> ㄴ


>>  2 번째 응답에 대한 MBTI 예측
1/1 [==============================] - 0s 31ms/step
  에측값 :  0.9710767865180969


   >> 97.11% 정도로 E 유형입니다.


#### 사용자 응답기반 MBTI 예측 - 추가질문.ver
- 예측값이 일정 확률이상 출력되지 않는다면 **1개의 추가 질문**을 한다.</br>
(0.2<예측값<0.8 일때 정확성이 떨어진다 판단)
</br></br>
추가질문을 하는 이유는 최대한 확실한 사용자의 MBTI를 도출하기 위함이다. </br>
MBTI의 E, I유형을 판단한다고 가정했을 때, 두 case로 나뉠 수 있다.</br>

 1. 본인이 90% 정도 E인 case</br>
 2. 본인이 64% 정도 E인 case</br>

 </br>

 **case 1.**의 경우는 본인이 MBTI가 극명한 E임에도 위의 **'주관식 MBTI 검사'**를 했을 때, 60%정도로 E가 나올 수도 있다. 이는 <기존의 객관식 검사에 비해 적은 양의 질문에 응답함, 학습데이터 수집의 한계로 인한 모델의 낮은 예측성> 등을 원인으로 들 수 있다. 이러한 문제점을 조금이나마 개선하기 위해 한 번의 추가 검사를 진행하도록 한다.
</br>
추가 질문에 응답했음에도 본인의 MBTI가 어느 한쪽 유형에 명확하게 나타나지 않는다면 </br>case 2.에 해당 될 것이다. 이러한 경우에는 본래 사용자의 MBTI 경계가 불분명한 것으로 판단하여 다른 질문을 추가로 하지않고 검사를 종료한다.
</br></br></br>
위의 설명은 가정에 불과하며, 추후 수정될 수 있음을 미리 알린다. 더불어 빠른 검사를 위해 일부러 검사시 사용되는 질문의 개수를 적게 설정하였다.


In [ ]:
#질문에 따른 응답입력 - 추가질문 버전

check_mbti = False
#실제 사용자 질의응답 단계
for n in range(2):
  similar = 0

  while similar < 1:
    #질의응답
    Question_Answer(n,qa_df)
    similar = cosine_similar(qa_df['Question'][n],qa_df['Answer'][n])
    print('유사도 : ', similar)

    if similar == 1:
      #예측 실행
      print('>> ', n+1, '번째 응답에 대한 MBTI 예측')
      MBTI = predict_sentence(loaded_model, str(qa_df['Answer'][n]), max_len)
      qa_df['MBTI'][n] = MBTI
      print('\n')
    else:
        print('다시~')

mbti_print_1(qa_df)

#추가질문
if check_mbti==True:
    #질의응답
    Question_Answer(2, qa_df)

    #예측 실행
    print('>> ', 3, '번째 응답에 대한 MBTI 예측')
    MBTI = predict_sentence(loaded_model, str(qa_df['Answer'][2]), max_len)
    qa_df['MBTI'][2] = MBTI
    print('\n')

    mbti_print_2(qa_df)